# Chapter 4 - 텍스트와 바이트
* 문자, 코드 포인트, 바인드 표현
* 이진 시퀀스의 고유한 특징: byte, bytearray, memoryview
* 전체 유니코드 및 레거시 문자셋에 대한 코덱
* 인코딩 에러를 피하고 다루기
* 텍스트 파일을 다룰 때의 모범 사례
* 기본 인코딩 및 표준 입출력 문제
* 정규화를 이용한 안전한 유니코드 텍스트 비교
* 정규화, 케이스 폴딩, 발음 구별 기호 강제 제거를 위한 유틸리티 함수
* locale과 PyUCA 라이브러리를 이용한 유니코드 텍스트의 적절한 정렬
* 유니코드 데이터베이스 안의 문자 메타데이터
* str과 bytes를 다루는 이중 모드 API

## 4.1 문자 문제

In [2]:
s = 'café'
len(s)

4

In [3]:
b = s.encode('utf8'); b

b'caf\xc3\xa9'

In [4]:
len(b)

5

In [5]:
b.decode('utf8')

'café'

## 4.2 바이트에 대한 기본 지식

In [7]:
cafe = bytes('café', encoding='utf-8'); cafe

b'caf\xc3\xa9'

In [8]:
cafe[0]

99

In [9]:
cafe[:1]

b'c'

In [11]:
cafe_arr = bytearray(cafe); cafe_arr

bytearray(b'caf\xc3\xa9')

In [12]:
cafe_arr[-1:]

bytearray(b'\xa9')

* b[0]은 range(256)에 들어가는 정수
* b[:1]은 bytes는 슬라이싱해도 bytes
* bytearray는 슬라이싱해도 bytearray

##### s[0] == s[:1] 이 되는 시퀀스형은 str이 유일하다.

In [13]:
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

In [14]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])
octets = bytes(numbers); octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

##### 'h' 타입코드는 short int(16비트) 형의 배열을 생성한다.

## 4.2.1 구조체와 메모리 뷰

In [19]:
import struct
fmt = '<3s3sHH'
with open('google.gif', 'rb') as fp:
    img = memoryview(fp.read())

In [20]:
header = img[:10]

In [21]:
bytes(header)

b'GIF89a|\x02f\x01'

In [22]:
struct.unpack(fmt, header)

(b'GIF', b'89a', 636, 358)

In [23]:
del header
del img

## 4.3 기본 인코더/디코더

In [24]:
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


##### 유니코드에서는 utf8(utf16)이 짱.

## 4.4 인코딩/디코딩 문제 이해하기

## 4.4.1 UnicodeEncodeError 처리하기

In [26]:
city = 'São Paulo'
city.encode('utf_8')

b'S\xc3\xa3o Paulo'

In [27]:
city.encode('utf_16')

b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [29]:
city.encode('iso8859_1')

b'S\xe3o Paulo'

In [31]:
city.encode('cp437')

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [32]:
city.encode('cp437', errors='ignore')

b'So Paulo'

In [33]:
city.encode('cp437', errors='replace')

b'S?o Paulo'

In [34]:
city.encode('cp437', errors='xmlcharrefreplace')

b'S&#227;o Paulo'

* utf-? 시리즈는 모든 str을 처리할 수 있다.
* encode에 errors 파라미터를 이용해서 에러를 '무시', '교체', 'xml형태로변환' 할 수 있다.

In [35]:
import codecs
codecs.register_error("handler", lambda x: ('z', x.end))
city.encode('cp437', errors='handler')

b'Szo Paulo'

##### codecs.register_error를 이용하여, 에러가 나는 부분에 대해 확장 할 수 있다.

## 4.4.2 UnicodeDecodeError 처리하기

In [43]:
a = 'Mentréal'.encode('utf-8'); a

b'Mentr\xc3\xa9al'

In [44]:
a.decode('utf-8')

'Mentréal'

##### utf-8은 인코딩만 제대로 되면 위와 같이 처리가 되나 utf-8로 인코딩이 안된 경우, 아래와 같이 에러를 발생한다.

In [45]:
octets = b'Mentr\xe9al'
octets.decode('cp1252')

'Mentréal'

In [46]:
octets.decode('iso8859_7')

'Mentrιal'

In [47]:
octets.decode('koi8_r')

'MentrИal'

In [48]:
octets.decode('utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [49]:
octets.decode('utf-8', errors='replace')

'Mentr�al'

## 4.4.3 예상과 달리 인코딩된 모듈을 로딩할 때 발생하는 SyntaxError

##### 파이썬은 2.5부터는 아스키, 파이썬 3부터는 utf-8을 기본 인코딩으로 사용했다.

##### 파이썬 3에서 인코딩 선언 없이 비utf-8로 인코딩된 경우 아래와 같은 아래가 발생함.

SyntaxError: Non-UTF-8 code starting with '\xef in fiLe oLa.py on line 1, but no encoding declared; see http://python.orgldev/peps/pep-0263/ for details

##### 이것에 대한 해결책은 아래의 coding 주석을 통해서 해결할 수 있다.

In [51]:
# coding: cp1252

print('Olá Mundo')

Olá Mundo


##### 하지만 대세는 utf-8이므로 utf-8을 사용하자!!

## 4.4.4 바이트 시퀀스의 인코딩 방식을 알아내는 방법


###### Chardet(Character - The Universal Character Encoding Detector)를 이용하자.....

## 4.4.5 BOM: 유용한 깨진 문자

##### 에디언 문제는 한 바이트 이상을 사용하는 utf_16과 utf_32에만 영향을 준다.

## 4.5 텍스트 파일 다루기

In [52]:
open('cafe.txt', 'w', encoding='utf-8').write('café')

4

In [54]:
open('cafe.txt').read()

'café'

##### window의 경우 기본 인코딩이 window 1252이므로, 에러가 발생하나, macOs Or Linux의 경우 기본 encoding이 utf-8 이므로 에러가 발생하지 않는다.

In [55]:
import os
os.stat('cafe.txt').st_size

5

In [57]:
fp2 = open('cafe.txt'); fp2

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='UTF-8'>

In [58]:
fp3 = open('cafe.txt', encoding='utf_8'); fp3

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='utf_8'>

##### fp2와 fp3에서 fp3이 더 좋은 방법이다. fp2 < fp3 

## 4.5.1 기본 인코딩 설정: 정신 나간 거 아냐?

In [1]:
import sys, locale

expressions = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
    """

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'UTF-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


##### 기본 인코딩에 의존하지 말자!
##### 프로그램 안에서 인코딩 설정을 하면, 많은 문제를 피할 수 있다.

## 4.6 제대로 비교하기 위해 유니코드 정규화하기

In [2]:
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [3]:
len(s1), len(s2)

(4, 5)

In [4]:
s1 == s2

False

In [5]:
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\u0301'
len(s1), len(s2)

(4, 5)

In [6]:
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [7]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [8]:
normalize('NFC', s1) == normalize('NFC', s2)

True

In [9]:
normalize('NFD', s1) == normalize('NFD', s2)

True

##### 안전을 보장하기 위해, 파일에 저장하기 전에 normalize('NFC', text) 코드로 문자열을 청소하는게 좋다.

In [10]:
from unicodedata import normalize, name
ohm = '\u2126'
name(ohm)

'OHM SIGN'